In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from tqdm import tqdm
import seaborn as sns
from IPython.display import display, Markdown
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import geopandas as gpd
import pandas as pd
import matplotlib.colors as mcolors
import imageio.v2 as imageio
import os
import tempfile
from PIL import Image, ImageDraw, ImageFont
from IPython.display import HTML
from scipy.optimize import minimize
from scipy.integrate import odeint
import pickle

In [2]:
with open("Proper_London_Patch_Populations_1_5km_fixed.pkl", "rb") as file:
    pop_dict = pickle.load(file)

small_pop_vector = np.array(list(pop_dict.values()))
pop_vector = np.zeros(63001)  # Create a zero array of the desired size

# Assign values at the correct positions
pop_vector[np.arange(251) * (251 + 1)] = small_pop_vector


with open("London Borough List.pkl", 'rb') as f:
    borough_list = pickle.load(f)
    
with open("London Station List.pkl", "rb") as file:
    station_list = pickle.load(file)
    
with open("London Subpopulation List.pkl", 'rb') as f:
    subpop_list = pickle.load(f)
    
with open("London Completed Commuter Matrix.pkl", "rb") as file:
    completed_commuter_matrix = pickle.load(file)

with open("London Home Patch Indices.pkl", "rb") as file:
    home_patches = pickle.load(file)
    
with open("London Travel Patch Indices.pkl", "rb") as file:
    travel_patches = pickle.load(file)

with open("London Travel Patch Names.pkl", "rb") as file:
    travel_patch_names = pickle.load(file)
    
with open("London new Nj.pkl", "rb") as file:
    Nj = pickle.load(file)

with open("London N=0 Indices Set.pkl", "rb") as file:
    N_0_set = pickle.load(file)

with open("London Station Borough List.pkl", 'rb') as f:
    station_borough_list = pickle.load(f)

with open("London Unique Patches.pkl", 'rb') as f:
    valid_patches = pickle.load(f)
    
with open("London Valid Patch Indices.pkl", 'rb') as f:
    valid_patch_indices = pickle.load(f)

with open("London Nonempty Subpopulation Indices.pkl", 'rb') as f:
    nonempty_subpop_indices = pickle.load(f)

with open("London Travel Patch Names.pkl", "rb") as file:
    travel_patch_names = pickle.load(file)

with open("final_best_beta_with_R.pkl", "rb") as file:
    betas = pickle.load(file)
    
with open("London P Populations.pkl", "rb") as file:
    p_populations = pickle.load(file)
    
covid_df = pd.read_csv("phe_cases_london_boroughs_fixed.csv")

borough_populations = pd.read_csv("london population 1801 to 2021_cleaned.csv")

london_gdf = gpd.read_file("Complete London Shapefile.gpkg", layer="Combined_London_Data")

london_commuters = pd.read_csv("London Commuters.csv").dropna(axis=1, how='all').dropna(axis=0, how='all')

london_tickets = pd.read_csv("London Ticket Data.csv").dropna(axis=0, how='all').dropna(axis=1, how='all')

In [3]:
with open("London Formatted Paths.pkl", "rb") as file:
    paths = pickle.load(file)

flat_paths = paths.flatten()
flat_paths.shape,flat_paths[1]

((63001,),
 [('Acton Town', 'Acton Town'),
  ('Acton Town', 'Turnham Green'),
  ('Turnham Green', 'Hammersmith'),
  ('Hammersmith', 'Barons Court'),
  ('Barons Court', "Earl's Court"),
  ("Earl's Court", 'Gloucester Road'),
  ('Gloucester Road', 'South Kensington'),
  ('South Kensington', 'Knightsbridge'),
  ('Knightsbridge', 'Hyde Park Corner'),
  ('Hyde Park Corner', 'Green Park'),
  ('Green Park', 'Piccadilly Circus'),
  ('Piccadilly Circus', 'Charing Cross'),
  ('Charing Cross', 'Embankment'),
  ('Embankment', 'Waterloo'),
  ('Waterloo', 'Bank & Monument'),
  ('Bank & Monument', 'Tower Hill'),
  ('Tower Hill', 'Aldgate'),
  ('Aldgate', 'Aldgate'),
  ('Turnham Green', 'Acton Town'),
  ('Hammersmith', 'Turnham Green'),
  ('Barons Court', 'Hammersmith'),
  ("Earl's Court", 'Barons Court'),
  ('Gloucester Road', "Earl's Court"),
  ('South Kensington', 'Gloucester Road'),
  ('Knightsbridge', 'South Kensington'),
  ('Hyde Park Corner', 'Knightsbridge'),
  ('Green Park', 'Hyde Park Corner

In [4]:
long_station_list = []
for i in range(251):
    for j in range(251):
        long_station_list.append(station_list[i])
long_station_list

['Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton Town',
 'Acton To

In [5]:
with open("London Boolean Array.pkl", "rb") as file:
    boolean_arr = pickle.load(file).astype(int)
    
boolean_arr

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [6]:
with open("combined_travel_times.pkl", "rb") as file:
    travel_times = pickle.load(file)
travel_times

{('Brixton', 'Stockwell'): 2,
 ('Stockwell', 'Vauxhall'): 2,
 ('Vauxhall', 'Pimlico'): 1,
 ('Pimlico', 'Victoria'): 2,
 ('Victoria', 'Green Park'): 2,
 ('Green Park', 'Oxford Circus'): 2,
 ('Oxford Circus', 'Warren Street'): 2,
 ('Warren Street', 'Euston'): 1,
 ('Euston', "King's Cross St. Pancras"): 2,
 ("King's Cross St. Pancras", 'Highbury & Islington'): 3,
 ('Highbury & Islington', 'Finsbury Park'): 2,
 ('Finsbury Park', 'Seven Sisters'): 4,
 ('Seven Sisters', 'Tottenham Hale'): 2,
 ('Tottenham Hale', 'Blackhorse Road'): 2,
 ('Blackhorse Road', 'Walthamstow Central'): 2,
 ('Stockwell', 'Brixton'): 2,
 ('Vauxhall', 'Stockwell'): 2,
 ('Pimlico', 'Vauxhall'): 1,
 ('Victoria', 'Pimlico'): 2,
 ('Green Park', 'Victoria'): 2,
 ('Oxford Circus', 'Green Park'): 2,
 ('Warren Street', 'Oxford Circus'): 2,
 ('Euston', 'Warren Street'): 1,
 ("King's Cross St. Pancras", 'Euston'): 2,
 ('Highbury & Islington', "King's Cross St. Pancras"): 3,
 ('Finsbury Park', 'Highbury & Islington'): 2,
 ('Seven

In [20]:
fourth_phase

array([[1345.2709    ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   8.63432903,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   8.19027465,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,   30.48144931],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    1.83627333],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1351.2195    ]])

In [ ]:
from scipy.sparse import csr_matrix

def convert_back_to_classic_p(fourth_phase_sparse):
    fifth_phase = np.zeros((63001, 63001))

    fourth_phase_coo = fourth_phase_sparse.tocoo()
    for i, val in zip(fourth_phase_coo.row, fourth_phase_coo.data):
        connection = connections_list[i]
        home_patch = connection[0]
        home_index = connections_list.index((home_patch, home_patch))
        fifth_phase[home_index] += val

    sixth_phase = np.zeros((63001, 63001))
    row_sums = fifth_phase.sum(axis=1)

    for i in range(63001):
        if row_sums[i] != 0:
            sixth_phase[i] = fifth_phase[i] / row_sums[i]
        else:
            sixth_phase[i][i] = 1

    return sixth_phase

isolated = convert_back_to_classic_p(sparse_fourth_phase)
isolated

In [24]:
fifth_phase = convert_back_to_classic_p(fourth_phase)
fifth_phase

KeyboardInterrupt: 

In [ ]:
def convert_back_to_classic_p(fourth_phase):
    home_indices = np.array([
        connections_list.index((conn[0], conn[0])) for conn in connections_list
    ])
    
    fifth_phase = np.zeros((63001, 63001))
    np.add.at(fifth_phase, (home_indices, slice(None)), fourth_phase)

    row_sums = fifth_phase.sum(axis=1, keepdims=True)
    sixth_phase = np.divide(
        fifth_phase, row_sums, out=np.eye(63001), where=row_sums != 0
    )

    return sixth_phase

maybe = convert_back_to_classic_p(fourth_phase)

In [ ]:
maybe

In [7]:
connections_list = []
for i in range(251):
    for j in range(251):
        connections_list.append((station_list[i],station_list[j]))

travel_vector = np.zeros(63001)
for i in range(63001):
    current_leg = connections_list[i]
    if current_leg[0]==current_leg[1]:
        travel_vector[i] = 8/24
    else:
        if current_leg in list(travel_times.keys()):
            travel_vector[i] = travel_times[current_leg]/(60*24)


In [8]:
myarr1 = np.array([
    [1,0,1],
    [0,1,1],
    [1,0,0]
])

values = np.array([
    3,4,5
])

(myarr1*values)

array([[3, 0, 5],
       [0, 4, 5],
       [3, 0, 0]])

In [8]:
second_phase = boolean_arr*travel_vector

In [9]:
# put remaining time into home
third_phase = second_phase.copy()
for i in range(63001):
    connection = connections_list[i]
    origin_station = connection[0]
    third_phase[i][connections_list.index((origin_station,origin_station))] += 1-np.sum(third_phase[i])

In [11]:
third_phase

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.61597222, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.61597222, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.61805556],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.58888889],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [10]:
with open("London P Populations.pkl", "rb") as file:
    p_populations = pickle.load(file).flatten()
p_populations

array([1345.2709,   14.0174,   13.2965, ...,   49.3183,    3.1182,
       1351.2195])

In [ ]:
fourth_phase = (third_phase.T*p_populations).T
fourth_phase

In [ ]:
import scipy.sparse as sp
sparse_fourth_phase = sp.csr_matrix(fourth_phase)

In [19]:
# with open("Sparse London Expanded P Matrix.pkl", "wb") as f:
    # pickle.dump(sparse_fourth_phase, f)

In [13]:
fourth_phase

array([[1345.2709    ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   8.63432903,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   8.19027465,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,   30.48144931],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    1.83627333],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1351.2195    ]])

In [15]:
def convert_back_to_classic_p(fourth_phase):
    fifth_phase = np.zeros((63001,63001))

    for i in range(63001):
        connection = connections_list[i]
        home_patch = connection[0]
        home_index = connections_list.index((home_patch,home_patch))
        fifth_phase[home_index] += fourth_phase[i]

    sixth_phase = np.zeros((63001,63001))

    for i in range(63001):
        if np.sum(fifth_phase[i]) != 0:
            sixth_phase[i] = fifth_phase[i]/np.sum(fifth_phase[i])
        else:
            sixth_phase[i][i] = 1
    
    return sixth_phase

In [23]:
from scipy.sparse import lil_matrix, csr_matrix

def build_fifth_phase_sparse(fourth_phase_sparse, connections_list):
    n = len(connections_list)
    result = lil_matrix((n, n))

    for i, (home, work) in enumerate(connections_list):
        if home == work:
            result[home, :] += fourth_phase_sparse[i].toarray().ravel()

    return result.tocsr()





build_fifth_phase_sparse(sparse_fourth_phase,connections_list).toarray()

IndexError: Index dimension must be <= 2

In [16]:
fifth_phase = np.zeros((63001,63001))

for i in range(63001):
    connection = connections_list[i]
    home_patch = connection[0]
    home_index = connections_list.index((home_patch,home_patch))
    fifth_phase[home_index] += fourth_phase[i]
    
fifth_phase

array([[6.42608466e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.61216667e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.14046667e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.92141443e+03]])

In [15]:
fifth_phase = np.zeros((63001,63001))

for i in range(63001):
    connection = connections_list[i]
    home_patch = connection[0]
    home_index = connections_list.index((home_patch,home_patch))
    fifth_phase[home_index] += fourth_phase[i]
    
fifth_phase

array([[6.42608466e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.61216667e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.14046667e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.92141443e+03]])

In [16]:
sixth_phase = np.zeros((63001,63001))

for i in range(63001):
    if np.sum(fifth_phase[i]) != 0:
        sixth_phase[i] = fifth_phase[i]/np.sum(fifth_phase[i])
    else:
        sixth_phase[i][i] = 1
        
sixth_phase

array([[6.87347937e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.72440839e-04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.42953833e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.94151417e-01]])

In [17]:
sixth_phase

array([[6.87347937e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.72440839e-04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.42953833e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.94151417e-01]])

In [18]:
# with open("London P Matrix.pkl", "wb") as f:
    # pickle.dump(sixth_phase, f)

KeyboardInterrupt: 

In [19]:
import scipy.sparse as sp

# Convert to sparse matrix
sparse_sixth_phase = sp.csr_matrix(sixth_phase)

# Save with pickle
# with open("Sparse London P Matrix.pkl", "wb") as f:
    # pickle.dump(sparse_sixth_phase, f)

In [20]:
# Load files
with open("London Condensed P Matrix.pkl", "rb") as file:
    sparse_p = pickle.load(file)

p_matrix = sparse_p.toarray()

In [23]:
diff = (sparse_p-sparse_sixth_phase).toarray()
diff

array([[ 0.11748478,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00017244],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00044295,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.10607319]])

In [20]:
np.any(sixth_phase < 0)


np.False_

In [24]:
np.max(diff),np.min(diff)

(np.float64(0.1511151877735829), np.float64(-0.03847370481107267))